# Ramsey King
# DSC 540 - Data Preparation
# Weeks 5 & 6
# July 16, 2021
## For this assignment you need to complete 8 of the following exercises against this data.  You must select at least 2 methods from Chapters 7, 8, 10 & 11.





In [5]:
import matplotlib.pyplot as plt
import pandas as pd

candy_2016_df = pd.read_excel('BOING-BOING-CANDY-HIERARCHY-2016-SURVEY-Responses.xlsx')
candy_2016_df.shape

(1259, 123)

### The 2 method from Chapter 7 that I will chose are filtering out missing data and binning. This will be done by filtering out the rows where the column 'How old are you?' is non-numeric.  From there, the ages will be binned.



In [22]:
# rename the 'How old are you?' column to 'age'
candy_2016_df = candy_2016_df.rename(columns={candy_2016_df.columns[3]:'age'})

# remove rows where 'age' column does not contain a number
candy_2016_df = candy_2016_df[pd.to_numeric(candy_2016_df['age'], errors='coerce').notnull()]
candy_2016_df.shape

(1191, 123)

In [37]:
# bin 'age' column based on age ranges (<17, 18-25, 26-34, 35-44, 45-54, 55-64, >65)
bins = [0,17,24,34,44,54,64,140]
cats = pd.cut(candy_2016_df['age'], bins)
pd.value_counts(cats)
# candy_2016_df.plot.bar(x=cats,y=pd.value_counts(cats))

(34, 44]     382
(24, 34]     317
(44, 54]     292
(54, 64]     121
(17, 24]      37
(0, 17]       21
(64, 140]     19
Name: age, dtype: int64

### The 2 methods that I will chose from Chapter 8 are combining/merging datasets and pivoting data.  This will be acheived as follows:
1. Create a subset of the 2015 and 2016 candy datasets based on all participants who gave a response in the '[100 Grand Bar]' column
2. Merge these subset datasets together (adding the rows of the 2016 dataset to the 2015 dataset).
3. Creating a pivot table based on the 100 Grand Bar response type by gender

In [42]:
# Load the 2015 and 2016 datasets (since I've re-written over the 2016 dataset above)
candy_2015_df = pd.read_excel('CANDY-HIERARCHY-2015-SURVEY-Responses.xlsx')
candy_2016_df = pd.read_excel('BOING-BOING-CANDY-HIERARCHY-2016-SURVEY-Responses.xlsx')

print("2015 dimensions:", candy_2015_df.shape, "\n2016 dimensions:", candy_2016_df.shape)

2015 dimensions: (5630, 124) 
2016 dimensions: (1259, 123)


In [49]:
# rename the 100 Grand Bar column '100_grand_bar'
candy_2015_df = candy_2015_df.rename(columns={candy_2015_df.columns[4]:'hundred_grand_bar'})
candy_2015_df.columns

# check for empty values in the 100 Grand Bar column in the 2015 dataset.
miss = candy_2015_df['hundred_grand_bar'].isnull().sum()
if miss>0:
    print("100 Grand Bar has {} missing value(s)".format(miss))
else:
    print("100 Grand Bar has no missing values.")

100 Grand Bar has 656 missing value(s)


In [52]:
# remove rows where the 100 Grand Bar value is null

candy_2015_df = candy_2015_df[candy_2015_df['hundred_grand_bar'].isnull() == False]
candy_2015_df.shape


656
